In [3]:
def metrics(y_true, y_score, lift = None, return_str = False):
    import sklearn
    import collections
    
    if True:
        # y_true = [0 if e == '-1' else 1 for e in os.popen('''perl -ne 'print substr($_,0,2) . "\n"' /data1/share/kosm/data/url_text_valid_vw.txt''').read().split()]
        # y_score = [float(e.strip('\n')) for e in open('/data1/share/kosm/data/url_text_valid_predictions_quadr.txt','r').readlines()]
        
        res = collections.OrderedDict()
        samp_size = len(y_true)
        res['Sample size'] = samp_size
        res['Posit share'] = sum(y_true) * 1./ samp_size
        res['Sample size'] = len(y_true)
        res['AUC ROC'] = sklearn.metrics.roc_auc_score(y_true = y_true, y_score = y_score)
        res['AUC PR'] = sklearn.metrics.auc(
                        *sklearn.metrics.precision_recall_curve(y_true = y_true, probas_pred  = y_score)[:2],
                        reorder = True
        )
        res['Log loss'] = sklearn.metrics.log_loss(y_true = y_true, y_pred = y_score)
        if lift:
            predictions_and_labels = sorted(zip(y_score,y_true), reverse = True)
            for l in lift:
                res['Lift ' + str(l)] = sum([e[1] for e in predictions_and_labels[:int(l * samp_size)]]) * 1. / int(l * samp_size) / res['Posit share']
                
        if return_str:
            res = '\n'.join(['{:<12}: {:.5f}'.format(k,v) for (k,v) in res.items()]) + '.'
        return res

In [6]:
#! head ./data/la_hll_test_scoring.tsv
data_str = [e.split('\t') for e in open('./data/la_hll_test_scoring.tsv','r').readlines()]
y_true  = [int(e[0]) for e in data_str]
y_score = [float(e[1]) for e in data_str]

m = metrics(y_true, y_score, lift = [0.0001,0.001, 0.01,0.05,0.1,0.2,0.5], return_str = True)

In [7]:
print(m)

Sample size : 67199894.00000
Posit share : 0.00001
AUC ROC     : 0.83151
AUC PR      : 0.00157
Log loss    : 0.05197
Lift 0.0001 : 413.59473
Lift 0.001  : 88.34704
Lift 0.01   : 23.68424
Lift 0.05   : 8.75940
Lift 0.1    : 5.90226
Lift 0.2    : 3.43985
Lift 0.5    : 1.79699.


In [27]:
sum(y_score),len(y_score)

(2755265.0491150017, 55299)